# Evaluation
This notebook contains code to reproduce the results presented in the Energy Informatics Open Journal. 

In [ ]:
import random, yaml
import math
import warnings
import math
import numpy as np
import pandas as pd
import pandapower
import inspect
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox
from pathlib import Path

from threats2power.cyber.analysis import Analyzer
from threats2power.communication.network import CommNetwork
from threats2power.attackers.random_attacker import RandomAttacker
from experiment import run_experiment
budget_dir = Path.cwd() / "data" / "results" / "budget"

## Varied Budget

### No Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 0.1 1 5 10 50 100 --child-no-deviation 0 --child-per-parent 999 --repeat_attacks True  --savename Budget-NoRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

### Some Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 0.1 1 5 10 50 100 --child-no-deviation 0 --child-per-parent 5 --repeat_attacks True  --savename Budget-SomeRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

### Max Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 0.1 1 5 10 50 100 --child-no-deviation 0 --child-per-parent 2 --repeat_attacks True  --savename Budget-MaxRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

## Fixed Budget

### No Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 52.0 --child-no-deviation 0 --child-per-parent 999 --repeat_attacks True  --savename FixedBudget-MaxRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

#### Some Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 52.0 --child-no-deviation 0 --child-per-parent 5 --repeat_attacks True  --savename FixedBudget-SomeRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

### Max Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 52.0 --child-no-deviation 0 --child-per-parent 2 --repeat_attacks True  --savename FixedBudget-MaxRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

## Random Budget

In [ ]:
from threats2power.cyber.criticality import criticality_by_capacity
from textwrap import wrap
def replot(name, save_name:str="Distribution"):
    exp_dir = budget_dir / name
    with open(exp_dir / f"{name}_metadata.yaml", "r") as stream:
        kwargs = yaml.load(stream, yaml.FullLoader)
    print("Metadata:\n\t", "\n\t ".join(wrap(", ".join([f"{k}:{v}" for k,v in kwargs.items()]), break_long_words=False, break_on_hyphens=False)))

    grid = kwargs.get("grid", "create_cigre_network_mv")
    grid_map = {name:creator for name, creator in inspect.getmembers(pandapower.networks, predicate=inspect.isfunction)}
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", category=FutureWarning)
        if type(grid) is str:
                grid = grid_map.get(grid)(**kwargs.get("grid_kwargs", {})) if grid is not None else None

    criticality=kwargs.get("criticality", criticality_by_capacity) 
    criticality = criticality(grid, verbose=False)[0] if criticality is not None else criticality
    network = CommNetwork(
        n_devices=kwargs.get("n_devices", 20),
        n_entrypoints=kwargs.get("n_entrypoints", 1),
        child_no_deviation=kwargs.get("child_no_deviation", 0),
        children_per_parent=kwargs.get("children_per_parent", 3),
        sibling_to_sibling_comm=kwargs.get("sibling_to_sibling_comm", None),
        repeated_attacks=kwargs.get("repeated_attacks", False),
        criticality=criticality,
        network_specs=Path.cwd() / "specifications" / "Default_specifications.json",
        grid=grid,
    )

    arrays = np.load(exp_dir / f"{name}.npz")
    compromised_array = arrays.get("compromise")
    effort_array = arrays.get("effort")
    criticality_array = arrays.get("criticality", np.zeros_like(compromised_array))
    print(f"Stored Shape: {compromised_array.shape}")

    analyzer = Analyzer(network)
    analyzer.res_monte = {**{"compromised":compromised_array, "effort":effort_array, "criticality":criticality_array}}
    analyzer.plot_monte(save_name=save_name, save_dir=exp_dir,
                        figsize=(14, 16) if "criticality" in arrays else (14,12), flatten=True)

### No Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 100000 --deviation 0 --per-parent 999 --param budget --values 0.0 100.0 --vary_entrypoints True --random_entry True --random_param True --repeat_attacks True --name Random-NoRedundancy --criticality capacity --sibling_to_sibling_comm all

In [ ]:
replot("Random-NoRedundancy")

### Some Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 100000 --deviation 0 --per-parent 5 --param budget --values 0.0 100.0 --vary_entrypoints True --random_entry True --random_param True --repeat_attacks True --name Random-SomeRedundancy --criticality capacity --sibling_to_sibling_comm all

In [ ]:
replot("Random-SomeRedundancy")

### Max Redundancy

In [ ]:
python ParallelAnalyzer.py --N 100000 --deviation 0 --per-parent 2 --param budget --values 0.0 100.0 --vary_entrypoints True --random_entry True --random_param True  --repeat_attacks True --name Random-MaxRedundancy --criticality capacity --sibling_to_sibling_comm all

In [ ]:
replot("Random-MaxRedundancy")

### Redundancy
Investigate the effect of redundancy (no. of children per parent) on the compromise and effort distribution of a communication network. 

In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="children_per_parent", param_values=[2, 3, 5, 8, 13, 21, 34, lambda network: network.n_devices],
               flatten=True, auto_compromise_children=False, save_name="Test",
               budget=52.0, repeated_attacks=True,
               n_attacks=1000)

### Budget
Investigate the effect of increasing the budget of attackers on the same communication network.
Effort Only means probability-of-success on defenses is ignored, if the effort threshold is met then that component is always compromised.


In [ ]:
!python ParallelAnalyzer.py --N 1000 --values 0.1 1 5 10 50 100 --repeat_attacks True --param budget --savename Budget-WithRepeatedAttacks --grid cigre

In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], # 250, 500, 1000, 2500, 5000, 10000
               children_per_parent = 3, save_name="Budget-NormalRedundancy", 
               repeated_attacks=True, flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)

### Susceptibility

In [ ]:
from experiment import run_experiment
from cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity

run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[52], save_name="Susceptibility",
               children_per_parent = 0, child_no_deviation=3, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True,
               criticality=criticality_by_capacity, 
               n_attacks=10000)

### Sibling-to-Sibling Communication


In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="sibling_to_sibling_comm", param_values=[False, "adjacent", "all"],
               vary_entrypoints=True,
               save_name="siblings",
               flatten=True,
               n_attacks=10000)

### Vary Entrypoints
Show effect of attacking from different entrypoints, using colorscale instead of a Legend.

In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[52], save_name="VariedEntrypoints",
               children_per_parent = 0, child_no_deviation=3, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=False,
               criticality=criticality_by_capacity, 
               n_attacks=10000)

### Static Analysis

#### Scalability

In [ ]:
import time
import seaborn as sns
from pathlib import Path
from tqdm.notebook import trange
from communication.network import CommNetwork
from cyber.analysis import Analyzer
REPETITIONS = 100
device_counts = [5,4,3,2]
spec_path = Path.cwd() / "specifications" / "Default_specifications.json"
time_taken = {}
for i in trange(REPETITIONS):
    for j, n_devices in enumerate(device_counts):
        network = CommNetwork(n_devices=n_devices, n_entrypoints=1, children_per_parent=np.random.randint(1,n_devices), child_no_deviation=np.random.randint(1,n_devices), sibling_to_sibling_comm="all",
                            network_specs=spec_path)
        n_comp = network.n_components
        analyzer = Analyzer(network)
        # Time how long static analysis takes to complete
        start = time.perf_counter()
        _ = analyzer.static_analysis(verbose=False)
        end = time.perf_counter()
        time_taken[n_comp] = [end-start] if n_comp not in time_taken else time_taken[n_comp] + [end-start]
        # print(f"No. of Devices: {n_devices}, Components: {n_comp}, Time Taken: {end-start}s")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

fig = plt.figure(figsize=(6,5))
avg_time_taken = pd.DataFrame([{i:np.mean(vals) for i,vals in time_taken.items()}])

sns.scatterplot(x=[0,1,2,3,4,5,6], y=avg_time_taken.T.sort_index()[0], marker='X', color="k", zorder=4, legend=None, ax=plt.gca())
sns.swarmplot(time_taken, size=2, log_scale=True, ax=plt.gca())
plt.gca().set(xlabel="No. of Components in Network", ylabel="Time Taken to Analyze (s)")
plt.tight_layout()
plt.savefig(Path.cwd() / "media" / "Scalability.pdf")

#### Comparison to Monte Carlo Approach

In [ ]:
network = CommNetwork(n_devices=3, n_entrypoints=1, children_per_parent=2, child_no_deviation=0, sibling_to_sibling_comm="all",
                      network_specs=spec_path)
analyzer = Analyzer(network)
static_distr = analyzer.static_analysis(verbose=False, show_paths=False)
attack_counts = [1, 10, 100, 1000, 10000] # Per Entrypoint
monte_distrs = {}
for i, n_attacks in enumerate(attack_counts):
    comp_distr, *_ = analyzer.monte_carlo_analysis(n_attacks=n_attacks, budget=999999999, device_only=False, vary_entrypoints=True)
    vals, counts = np.unique(comp_distr, return_counts=True)
    monte_distr = {val:count/np.sum(counts) for val, count in zip(vals, counts)}
    monte_distrs[n_attacks] = monte_distr

In [ ]:
df = []
for key, distr in monte_distrs.items():
    for n_comp, prob in distr.items():
        df.append([f"Monte w. {key} runs" if isinstance(key, int) else f"Static Analysis", n_comp, prob])
df = pd.DataFrame(df, columns=["Type", "Components", "Probs"])
fig = plt.figure(figsize=(6,5))
sns.barplot(df, x="Components", y="Probs", hue="Type", errorbar=None, gap=0)
plt.gca().set(xlabel="No. of Components Compromised", ylabel="Probability")
plt.tight_layout()
plt.savefig(Path.cwd() / "media" / "StaticAnalysis.pdf")
plt.show()

### Use on Real Grid

In [ ]:
from cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[52], save_name="ParallelRealGrid",
               children_per_parent = 32, child_no_deviation = 8,
               sibling_to_sibling_comm="all", vary_entrypoints=True, flatten=True,
               criticality=criticality_by_capacity, max_criticality=400.0, 
               n_attacks=1000)

In [ ]:
from cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], save_name="SpanishLVBudget",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)

In [ ]:
from cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], save_name="SpanishLVBudget",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)

In [ ]:
from cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], save_name="SpanishLVBudget",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)